In [5]:
# import libraries
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

# ML libraries
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [13]:
# get data and put it in a pandas dataframe

df = pd.read_csv("AoT_Chicago-Copy1.complete.recent.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45740 entries, 0 to 45739
Data columns (total 7 columns):
timestamp    45740 non-null object
node_id      45740 non-null object
subsystem    45740 non-null object
sensor       45740 non-null object
parameter    45740 non-null object
value_raw    43416 non-null object
value_hrf    43957 non-null object
dtypes: object(7)
memory usage: 2.4+ MB


,timestamp,node_id,parameter,value_hrf
0,2019/11/20 05:25:10,001e06117b41,temperature,6.7
1,2019/11/20 05:25:10,001e06117b41,temperature,7.09
2,2019/11/20 05:25:10,001e06117b41,temperature,8.02
3,2019/11/20 05:25:10,001e06117b41,temperature,8.67
4,2019/11/20 05:25:10,001e06117b41,id,5410ecf722f8
5,2019/11/20 05:25:10,001e06117b41,concentration,0.37253
6,2019/11/20 05:25:10,001e06117b41,concentration,-0.01153
7,2019/11/20 05:25:10,001e06117b41,pressure,996.41
8,2019/11/20 05:25:10,001e06117b41,temperature,8.2
9,2019/11/20 05:25:10,001e06117b41,concentration,0.01719


In [ ]:
# TODO: slice the data

df.drop(columns=["sensor","value_raw","subsystem"],axis=1)   # delete irrelevant data